In [1]:
#include "TFile.h"
#include "TH1.h"
#include "TH2.h"
#include "TString.h"
#include "TAttMarker.h"
#include "TLatex.h"
#include "TCanvas.h"
#include "TStyle.h"
#include "TGaxis.h"
#include "TTree.h"
#include "TRandom.h"  
#include "TClass.h"

#include <dirent.h>     //for locating directories
#include <math.h> 
#include <iostream>

#include "Clipped_Wave.C"


In [2]:
//template directory
TString template_file = "./templates/AD_Templates.root";

TFile *f = TFile::Open(template_file);
TGraph* graph = (TGraph*)f->Get("gPSD_Template_3_0");
//TGraph* graph = (TGraph*)f->Get("gPSD_Template_108_1");
f->Close();


In [3]:
//TH2F* PSDvE = new TH2F("PSD verses Energy", "", 200,0,200, 1000,0,1);
//TString PSDvE_file = "/home/austin/Documents/test/data/simulated/combined/AD1_Wet_MuonTracking.root";
//AddPlots(PSDvE_file, "hPSDvE", PSDvE);


// from graph, get maximum and gradually scale it
Int_t n; n = graph->GetN(); //get plotted array dimention
Double_t max_y = findMaxPosition(graph);
//Double_t cutoff= max_y*1.5; // when clipping takes place
Double_t cutoff= 20000;
Double_t dh_cutoff = max_y*40; // when to stop scaling
Double_t dh = -max_y+1.0;

In [4]:
// create a list of events to contrast as the wave scales by dh
struct compare { Double_t dh, PSD, area, unclipped_area,time, unclipped_time;};
vector<compare> datalist;

In [5]:
int counter = 0;
while (dh < dh_cutoff) {

    Double_t * ax = new Double_t[n];
    Double_t * ay = new Double_t[n];
    Double_t * ay_new = new Double_t[n];
    Double_t * ay_scaled = new Double_t[n];

    // scaling fraction
    double s = (max_y + dh) / max_y;
    for(Int_t i = 0; i < n; i++){
      graph->GetPoint(i,ax[i],ay[i]);

      ay_new[i] = ay[i]*s;
      ay_scaled[i] = ay[i]*s;
      if(ay_new[i] >= cutoff) ay_new[i] = cutoff;
    }

    TGraph* graph_new = new TGraph(n,ax,ay_new);
    TGraph* graph_scaled = new TGraph(n,ax,ay_scaled);

    delete [] ax;      delete [] ay;
    delete [] ay_new;  delete [] ay_scaled;

    counter++; 

    compare C;
    C.dh = dh; 
    C.PSD = Get_PSD(graph_new); 
    C.time = Get_Time(graph_new);
    C.unclipped_time = Get_Time(graph_scaled); 
    C.area = Get_Energy(graph_new); 
    C.unclipped_area = Get_Energy(graph_scaled);
    datalist.push_back(C);
    dh += 1000.0;

    cout << "working on dh"<< endl;
    
}


working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
working on dh
workin

Error in <GSLError>: Error 18 in qags.c at 547 : cannot reach tolerance because of roundoff error
Warning in <TF1::IntegralOneDim>: Error found in integrating function f1 in [43.931301,199.931301] using AdaptiveSingular. Result = 3.691243 +/- 0.000353  - status = 18
Info in <TF1::IntegralOneDim>: 		Function Parameters = {
Error in <GSLError>: Error 18 in qags.c at 547 : cannot reach tolerance because of roundoff error
Warning in <TF1::IntegralOneDim>: Error found in integrating function f1 in [-12.068699,199.931301] using AdaptiveSingular. Result = 28.594738 +/- 0.000668  - status = 18
Info in <TF1::IntegralOneDim>: 		Function Parameters = {
Error in <GSLError>: Error 18 in qags.c at 547 : cannot reach tolerance because of roundoff error
Warning in <TF1::IntegralOneDim>: Error found in integrating function f1 in [-50.000000,500.000000] using AdaptiveSingular. Result = 29.409169 +/- 0.003052  - status = 18
Info in <TF1::IntegralOneDim>: 		Function Parameters = {
Error in <GSLError>: Err

In [6]:
Double_t * ah = new Double_t[(int)datalist.size()];
Double_t * a_time = new Double_t[(int)datalist.size()];
Double_t * a_unclippedtime = new Double_t[(int)datalist.size()]; 
Double_t * aPSD = new Double_t[(int)datalist.size()];
Double_t * aE = new Double_t[(int)datalist.size()];
Double_t * aE_unclipped = new Double_t[(int)datalist.size()];

//double offset = list[0].area / 2000.0;
for(int i = 0; i < (int)datalist.size(); ++i){
ah[i] = datalist[i].dh;
aPSD[i] = datalist[i].PSD;
aE[i] = datalist[i].area; /// 2000.0 -offset;
a_time[i] = datalist[i].time - datalist[i].unclipped_time;
aE_unclipped[i] = datalist[i].unclipped_area;
}

// cout << "__________________________"<< endl;
// cout << "\n\n"<< endl;

In [7]:
TGraph* g_PSD_vs_dh = new TGraph((int)datalist.size(),ah,aPSD);
TGraph* g_area_vs_dh = new TGraph((int)datalist.size(),ah,aE);
TGraph* g_time_vs_dh = new TGraph((int)datalist.size(),ah,a_time);
TGraph* g_PSD_vs_area = new TGraph((int)datalist.size(),aE,aPSD);
TGraph* g_PSD_vs_time = new TGraph((int)datalist.size(), a_time, aPSD);
TGraph* g_time_vs_PSD = new TGraph((int)datalist.size(), aPSD, a_time);
//TGraph* Calculated_PSDvE = new TGraph((int)datalist.size, aE,aPSD);
g_copy = new TGraph((int)datalist.size(), aE,aPSD);
TF1 *f1 = new TF1("f1",myfunc,aE[0],aE[(int)datalist.size()-1]);

delete [] ah; delete [] aPSD; delete [] a_time;
delete [] aE; delete [] aE_unclipped;


In [8]:
%jsroot on
TCanvas *c1 = new TCanvas();
graph->SetTitle("Waveform Template Example");
graph->GetXaxis()->SetTitle("time [ns]");
graph->GetYaxis()->SetTitle("ADC units (arbitrary)");
graph->SetMarkerColor(4);
graph->Draw("AL");
c1->SetGridx();
c1->SetGridy();
c1->Draw("")

In [9]:
%jsroot on
TCanvas *c2 = new TCanvas();
g_PSD_vs_dh->GetXaxis()->SetTitle("#Deltah [arbitrary]");
g_PSD_vs_dh->GetYaxis()->SetTitle("PSD");
g_PSD_vs_dh->SetTitle("PSD vs Change in Height");
g_PSD_vs_dh->Draw();
c2->SetGridx();
c2->SetGridy();
c2->Draw();

In [10]:
%jsroot on
TCanvas *c3 = new TCanvas();
g_area_vs_dh->GetXaxis()->SetTitle("#Deltah [arbitrary]");
g_area_vs_dh->GetYaxis()->SetTitle("Area");
g_area_vs_dh->SetTitle("Area vs Change in Height");
g_area_vs_dh->Draw();
c3->SetGridx();
c3->SetGridy();
c3->Draw()

In [11]:
%jsroot on
TCanvas *c4 = new TCanvas();
g_PSD_vs_area->GetXaxis()->SetTitle("Area");
g_PSD_vs_area->GetYaxis()->SetTitle("PSD");
g_PSD_vs_area->SetTitle("PSD vs Clipped Waveform Area");
g_PSD_vs_area->GetYaxis()->SetRangeUser(0, 1);
g_PSD_vs_area->Draw();
c4->SetGridx();
c4->SetGridy();
c4->Draw(); 

In [23]:
%jsroot on
TCanvas *c5 = new TCanvas();
g_time_vs_dh->GetXaxis()->SetTitle("#Deltah [arbitrary]");
g_time_vs_dh->GetYaxis()->SetTitle("#Deltat [ns]");
g_time_vs_dh->SetTitle("time offset vs Change in Height");
g_time_vs_dh->Draw();
c5->SetGridx();
c5->SetGridy();
c5->Draw();

In [25]:
%jsroot on
TCanvas *c6 = new TCanvas();
g_time_vs_PSD->GetYaxis()->SetTitle("#Deltat [ns]");
g_time_vs_PSD->GetXaxis()->SetTitle("PSD");
g_time_vs_PSD->SetTitle("PSD vs time offset");
g_time_vs_PSD->Draw();
c6->SetGridx();
c6->SetGridy();
c6->Draw();

In [14]:

//   gStyle->SetOptStat(0);
//   TCanvas *c7 = new TCanvas();
//   PSDvE->SetTitle("PSD vs Energy");
//   PSDvE->GetYaxis()->SetTitle("PSD");
//   PSDvE->GetXaxis()->SetTitle("Enegy");
//   PSDvE->Draw();
//   g_PSD_vs_area->Draw("SAME");


TCanvas *c8 = new TCanvas();
f1->Draw();

c8->Draw();

## Debugging

In [15]:
// TFile *muonfile = TFile::Open("./AD1_Wet_MuonTracking.root");
// TH2F* PSDvE = (TH2F*)muonfile->Get("PSDvE");
// muonfile->Close();

In [16]:
void rescaleaxis_x(TGraph *g, double scale) { 
    int N = g->GetN(); 
    double *x = g->GetX(); 
    int i=0; while(i<N) { x[i]=x[i]*scale; i=i+1; } 
    g->GetHistogram()->Delete(); 
    g->SetHistogram(0); 
}

In [17]:
void rescaleaxis_y(TGraph *g, double scale) { 
    int N = g->GetN(); 
    double *y = g->GetY(); 
    int i=0; while(i<N) { y[i]=y[i]*scale; i=i+1; } 
    g->GetHistogram()->Delete(); 
    g->SetHistogram(0); 
}

In [18]:
void shiftaxis_x(TGraph *g, double shift) { 
    int N = g->GetN(); 
    double *x = g->GetX(); 
    int i=0; while(i<N) { x[i]=x[i] + shift; i=i+1; } 
    g->GetHistogram()->Delete(); 
    g->SetHistogram(0); 
}

In [19]:
void shiftaxis_y(TGraph *g, double shift) { 
    int N = g->GetN(); 
    double *y = g->GetY(); 
    int i=0; while(i<N) { y[i]=y[i] + shift; i=i+1; } 
    g->GetHistogram()->Delete(); 
    g->SetHistogram(0); 
}

In [20]:
TH2F* PSDvE = new TH2F("PSD verses Energy", "", 200,0,200, 200,0,1);
TH2* h;
TFile *muonfile = TFile::Open("./AD1_Wet_MuonTracking.root");
muonfile->cd("MuonTracking");
gDirectory->GetObject("hPSDvE",h);
PSDvE->Add(h);
muonfile->Close();



Warning in <TClass::Init>: no dictionary for class TCumulativeMap<string,double> is available
Warning in <TClass::Init>: no dictionary for class TCumulative is available
Warning in <TClass::Init>: no dictionary for class CumulativeData is available
Info in <TH2F::Add>: Attempt to add histograms with different number of bins - trying to use TH1::Merge


In [21]:
g_PSD_vs_area_scaled = (TGraph*)g_PSD_vs_area->Clone();
rescaleaxis_x(g_PSD_vs_area_scaled, 0.00005);
//shiftaxis_x(g_PSD_vs_area_scaled, 8.);
//shiftaxis_y(g_PSD_vs_area_scaled, 0.015);

In [26]:
TCanvas *c9 = new TCanvas();
PSDvE->Draw("colz");

PSDvE->GetXaxis()->SetTitle("Energy [MeV]");
PSDvE->GetYaxis()->SetTitle("PSD");
PSDvE->GetXaxis()->SetRangeUser(0, 150);

g_PSD_vs_area_scaled->Draw("L");
g_PSD_vs_area_scaled->SetLineColor(2);

c9->SetGridx();
c9->SetGridy();
c9->Draw();